In [1]:
from typing import List, Dict, Any, Optional, Sequence, Callable, Literal
from pydantic import BaseModel, Field, StrictFloat

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool
from langchain_ollama import ChatOllama

from langgraph.prebuilt import ToolNode
from langgraph.graph import StateGraph, MessagesState, START, END

from typing import List, Dict, Any, Optional, Sequence, Callable, Literal
from pydantic import BaseModel, Field, StrictFloat

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool
from langchain_ollama import ChatOllama

from langgraph.prebuilt import ToolNode
from langgraph.graph import StateGraph, MessagesState, START, END

In [2]:
import sys
sys.path.append(r"C:\Users\pasupuleti\Desktop\carla-drowsiness-detection")  # to import from parent directory

In [3]:
"""_summary_"""

import asyncio
import time
import logging
import threading
from pydantic import BaseModel, Field
from langchain.tools import tool
from src.Driver_assistance_bot.controls import VoiceControl, WheelControlVibration

voice = VoiceControl()
steering = WheelControlVibration()

# configure logging once at the start of your program
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s [%(levelname)s] %(message)s"
)



# ---------------- Vibrate Steering ----------------
class VibrateSteeringSchema(BaseModel):
    duration: float = Field(..., gt=0, le=3, description="Duration in seconds")
    intensity: int = Field(..., ge=0, le=60, description="Vibration intensity")


class VoiceAlertSchema(BaseModel):
    text: str = Field(..., description="Text to speak")





@tool(args_schema=VoiceAlertSchema)
async def voice_alert(text: str) -> str:
    """Alert the driver via text-to-speech asynchronously."""
    try:
        await voice.text_to_speech(text)
        return f"Voice alert completed: '{text}'"
    except Exception as e:
        logging.error(f"[Voice Alert Error] {e}")
        return f"Error: {e}"


@tool(args_schema=VibrateSteeringSchema)
async def vibrate_steering_wheel(duration: float, intensity: int) -> str:
    """Vibrate the steering wheel asynchronously for the given duration and intensity."""
    try:
        await steering.vibrate(duration, intensity)
        return (
            f"Steering wheel vibration completed: {duration}s at intensity {intensity}"
        )
    except Exception as e:
        logging.error(f"[Steering Wheel Error] {e}")
        return f"Error: {e}"

Failed to initialize Logitech G29 device logitech_raw. Vibration will not work.  Error: [Errno 2] No such file or directory: '/dev/logitech_raw'


In [4]:
class Bot:
    """Schemas for Driver Assistance Bot"""

    class BotConfig(BaseModel):
        model_id: str
        system_prompt: str
        tools: Sequence[Callable[..., Any]] | None = None
        temperature: float | None = None

    class Input(BaseModel):
        perclos: Optional[StrictFloat] = Field(
            default=None, description="Percentage of time eyes are closed."
        )
        blink_rate: Optional[StrictFloat] = Field(
            default=None, description="Number of eye blinks per minute."
        )
        yawn_freq: Optional[StrictFloat] = Field(
            default=None, description="Number of yawns per minute."
        )
        sdlp: Optional[StrictFloat] = Field(
            default=None, description="Standard deviation of lane position (m)."
        )

    class Output(BaseModel):
        drowsiness_level: Literal["low", "medium", "high", "critical"] = Field(
            description="Detected drowsiness risk level."
        )
        reasoning: str = Field(
            description="Explanation based on input metrics that led to the decision."
        )
        tool_calls: List[Dict[str, Any]] = Field(
            description="List of tool calls executed in response to drowsiness."
        )

In [5]:
# -------------------------------------------------------------------
# 3. Bot implementation with simplified LangGraph
# -------------------------------------------------------------------
class DriverAssistanceBot(Bot):
    def __init__(self, config: Bot.BotConfig):
        self.config = config

        # LLM model
        self.model = ChatOllama(model=config.model_id, temperature=config.temperature)

        # Bind tools
        self.model_with_tools = self.model.bind_tools(config.tools)

        # Tool node
        self.tool_node = ToolNode(config.tools)

        # Prompt template
        self.prompt = ChatPromptTemplate.from_messages(
            [("system", config.system_prompt)]
        )

        # Build graph
        builder = StateGraph(MessagesState)

        # Model call node
        def call_model(state: MessagesState):
            messages = state["messages"]
            response = self.model_with_tools.invoke(messages)
            return {"messages": [response]}  # avoid in-place mutation

        builder.add_node("call_model", call_model)
        builder.add_node("tools", self.tool_node)

        # Simple linear flow: start → model → tools → end
        builder.add_edge(START, "call_model")
        builder.add_edge("call_model", "tools")
        builder.add_edge("tools", END)  # no loopback, graph ends after tools

        self.graph = builder.compile()

    def invoke(self, input_data: Bot.Input):
        """Invoke the driver assistance bot with structured input metrics"""
        system_message = {"role": "system", "content": self.config.system_prompt}
        user_message = {
            "role": "user",
            "content": (
                f"Drowsiness metrics: {input_data.dict()}.\n"
                "Assess drowsiness risk, explain reasoning, "
                "and call appropriate alert tools."
            ),
        }

        result = self.graph.invoke({"messages": [system_message, user_message]})
        return result

In [6]:
# -------------------------------------------------------------------
# 5. Main function to run the bot
# -------------------------------------------------------------------
if __name__ == "__main__":
    # 1. Configure the bot
    config = Bot.BotConfig(
        model_id="llama3.1:8b",
        temperature=0,
        system_prompt=(
            "You are a driver assistance system. "
            "Analyze the provided drowsiness metrics and take appropriate action "
            "by using the available tools to alert the driver. "
            "A high PERCLOS value (e.g., > 0.3), high yawn count, or high SDLP "
            "indicates a serious risk. In such cases, use a high intensity steering wheel vibration "
            "AND a strong voice alert together. "
            "invoke both the tools"
        ),
        tools=[voice_alert, vibrate_steering_wheel],
    )

    # 2. Initialize the bot
    bot = DriverAssistanceBot(config)

    # 3. Example driver metrics
    metrics = Bot.Input(
        perclos=0.9,
        blink_rate=2,
        yawn_freq=4,
        sdlp=0.6,
    )

    # 4. Invoke the bot
    print("\n--- Invoking Driver Assistance Bot ---")
    response = bot.invoke(metrics)

    # 5. Print the output
    print("\n--- Bot Response ---")
    print(response)


--- Invoking Driver Assistance Bot ---


c:\temp\ipykernel_17456\2529423747.py:47: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  f"Drowsiness metrics: {input_data.dict()}.\n"
2025-08-27 10:12:50,272 [INFO] HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"



--- Bot Response ---
{'messages': [SystemMessage(content='You are a driver assistance system. Analyze the provided drowsiness metrics and take appropriate action by using the available tools to alert the driver. A high PERCLOS value (e.g., > 0.3), high yawn count, or high SDLP indicates a serious risk. In such cases, use a high intensity steering wheel vibration AND a strong voice alert together. invoke both the tools', additional_kwargs={}, response_metadata={}, id='6a7a72ee-b095-4aa6-810c-47c8fa46f6ac'), HumanMessage(content="Drowsiness metrics: {'perclos': 0.9, 'blink_rate': 2.0, 'yawn_freq': 4.0, 'sdlp': 0.6}.\nAssess drowsiness risk, explain reasoning, and call appropriate alert tools.", additional_kwargs={}, response_metadata={}, id='0095bf83-c3fc-4768-8b70-814ef19a284b'), AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.1:8b', 'created_at': '2025-08-27T08:12:53.7534742Z', 'done': True, 'done_reason': 'stop', 'total_duration': 38206414500, 'load_du

In [7]:
from pprint import pprint
pprint(response["messages"])

[SystemMessage(content='You are a driver assistance system. Analyze the provided drowsiness metrics and take appropriate action by using the available tools to alert the driver. A high PERCLOS value (e.g., > 0.3), high yawn count, or high SDLP indicates a serious risk. In such cases, use a high intensity steering wheel vibration AND a strong voice alert together. invoke both the tools', additional_kwargs={}, response_metadata={}, id='6a7a72ee-b095-4aa6-810c-47c8fa46f6ac'),
 HumanMessage(content="Drowsiness metrics: {'perclos': 0.9, 'blink_rate': 2.0, 'yawn_freq': 4.0, 'sdlp': 0.6}.\nAssess drowsiness risk, explain reasoning, and call appropriate alert tools.", additional_kwargs={}, response_metadata={}, id='0095bf83-c3fc-4768-8b70-814ef19a284b'),
 AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.1:8b', 'created_at': '2025-08-27T08:12:53.7534742Z', 'done': True, 'done_reason': 'stop', 'total_duration': 38206414500, 'load_duration': 8591182300, 'prompt_eval

In [19]:
model = ChatOllama(model=config.model_id, temperature=config.temperature)

In [38]:
def get_weather(location: str):
    """Call to get the current weather."""
    if location.lower() in ["sf", "san francisco"]:
        return "It's 60 degrees and foggy."
    else:
        return "It's 90 degrees and sunny."


def get_touristplaces(location: str):
    """Call to get the current tourist places."""
    if location.lower() in ["sf", "san francisco"]:
        return ["Golden Gate Park", "Alcatraz Island", "Fisherman's Wharf"]
    else:
        return ["Central Park", "Statue of Liberty", "Metropolitan Museum of Art"]


model_with_tools = model.bind_tools([get_weather, get_touristplaces])

In [39]:
input = {
    "messages": [
        {"role": "user", "content": "what's the weather and tourist places in sf?"}
    ]
}

In [40]:
output = model_with_tools.invoke(input["messages"])

2025-08-27 10:28:42,570 [INFO] HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


In [41]:
response = {"messages": [output]}

In [42]:
response["messages"][-1].tool_calls

[{'name': 'get_weather',
  'args': {'location': 'sf'},
  'id': 'ea4290ee-3c1d-4662-8b37-c49f2a3b2a3e',
  'type': 'tool_call'},
 {'name': 'get_touristplaces',
  'args': {'location': 'sf'},
  'id': '9e1867f9-c5d1-4737-8cb8-a801e6ae3a5f',
  'type': 'tool_call'}]

In [43]:
tool_node = ToolNode([get_weather, get_touristplaces])

In [47]:
tool_output = tool_node.invoke(response)
tool_output

{'messages': [ToolMessage(content="It's 60 degrees and foggy.", name='get_weather', tool_call_id='ea4290ee-3c1d-4662-8b37-c49f2a3b2a3e'),
  ToolMessage(content='["Golden Gate Park", "Alcatraz Island", "Fisherman\'s Wharf"]', name='get_touristplaces', tool_call_id='9e1867f9-c5d1-4737-8cb8-a801e6ae3a5f')]}

In [48]:
model_with_tools.invoke(tool_output["messages"])

2025-08-27 11:03:20,116 [INFO] HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


AIMessage(content='Based on the weather conditions, it would be best to visit indoor attractions in San Francisco. Considering the options provided, I would recommend visiting Golden Gate Park as it has several museums and gardens that are perfect for a foggy day. The park also offers a peaceful atmosphere, which is ideal for escaping the gloomy weather.', additional_kwargs={}, response_metadata={'model': 'llama3.1:8b', 'created_at': '2025-08-27T09:03:27.3345879Z', 'done': True, 'done_reason': 'stop', 'total_duration': 21637805900, 'load_duration': 9835417000, 'prompt_eval_count': 91, 'prompt_eval_duration': 4578611500, 'eval_count': 65, 'eval_duration': 7220710900, 'model_name': 'llama3.1:8b'}, id='run--f3e78196-fe56-4c8f-aa3c-9427f5c25f05-0', usage_metadata={'input_tokens': 91, 'output_tokens': 65, 'total_tokens': 156})

In [ ]:
from langgraph.prebuilt import ToolNode